## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.



## Connecting to the database

In [10]:
!pip uninstall prettytable -y
!pip install prettytable==2.5.0


Found existing installation: prettytable 3.17.0
Uninstalling prettytable-3.17.0:
  Successfully uninstalled prettytable-3.17.0


In [1]:
%load_ext sql

In [2]:
import csv, sqlite3

con = sqlite3.connect('my_data.db')
cur = con.cursor()

In [3]:
%sql sqlite:///my_data1.db

In [4]:
import pandas as pd
df = pd.read_csv("Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method='multi')

101

**Note:This below code is added to remove blank rows from table**


In [5]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


## Data Inspection

In [6]:
%config SqlMagic.autopandas = True

In [7]:
%%sql

SELECT * FROM SPACEXTBL LIMIT 5;

 * sqlite:///my_data1.db
Done.


,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [8]:
%%sql

SELECT DISTINCT(Launch_Site) FROM SPACEXTBL;

 * sqlite:///my_data1.db
Done.


,Launch_Site
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40


## Exploratory Data Analysis


**1- Displaying the names of the unique launch sites in the space mission**

In [9]:
%%sql

SELECT DISTINCT(Landing_Outcome) FROM SPACEXTBL;

 * sqlite:///my_data1.db
Done.


,Landing_Outcome
0,Failure (parachute)
1,No attempt
2,Uncontrolled (ocean)
3,Controlled (ocean)
4,Failure (drone ship)
5,Precluded (drone ship)
6,Success (ground pad)
7,Success (drone ship)
8,Success
9,Failure


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [10]:
%%sql

SELECT Launch_Site
FROM SPACEXTBL
WHERE Launch_Site LIKE 'CCA%'
LIMIT 5;

 * sqlite:///my_data1.db
Done.


,Launch_Site
0,CCAFS LC-40
1,CCAFS LC-40
2,CCAFS LC-40
3,CCAFS LC-40
4,CCAFS LC-40


##### Display the total payload mass carried by boosters launched by NASA (CRS)

In [11]:
%%sql

SELECT SUM(PAYLOAD_MASS__KG_) AS TOTAL_PAYLOAD_MASS
FROM SPACEXTBL
WHERE Customer = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


,TOTAL_PAYLOAD_MASS
0,45596


##### Display average payload mass carried by booster version F9 v1.1

In [12]:
%%sql

SELECT AVG(PAYLOAD_MASS__KG_) AS AVG_PAYLOAD_MASS
FROM SPACEXTBL
WHERE Booster_Version = 'F9 v1.1';

 * sqlite:///my_data1.db
Done.


,AVG_PAYLOAD_MASS
0,2928.4


##### List the date when the first succesful landing outcome in ground pad was acheived.

In [13]:
%%sql

SELECT MIN(Date) AS First_groundpad_landing
FROM SPACEXTBL
WHERE Landing_Outcome = 'Success (ground pad)';

 * sqlite:///my_data1.db
Done.


,First_groundpad_landing
0,2015-12-22


##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [14]:
%%sql

SELECT DISTINCT(Booster_Version)
FROM SPACEXTBL
WHERE (PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000) AND Landing_Outcome = 'Success (drone ship)';

 * sqlite:///my_data1.db
Done.


,Booster_Version
0,F9 FT B1022
1,F9 FT B1026
2,F9 FT B1021.2
3,F9 FT B1031.2


##### List the total number of successful and failure mission outcomes


In [15]:
%%sql

SELECT COUNT(Mission_Outcome) AS Total_outcomes
FROM SPACEXTBL;

 * sqlite:///my_data1.db
Done.


,Total_outcomes
0,101


##### List the   names of the booster_versions which have carried the maximum payload mass.


In [16]:
%%sql

SELECT Booster_Version
FROM SPACEXTBL
WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL);

 * sqlite:///my_data1.db
Done.


,Booster_Version
0,F9 B5 B1048.4
1,F9 B5 B1049.4
2,F9 B5 B1051.3
3,F9 B5 B1056.4
4,F9 B5 B1048.5
5,F9 B5 B1051.4
6,F9 B5 B1049.5
7,F9 B5 B1060.2
8,F9 B5 B1058.3
9,F9 B5 B1051.6


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**


In [17]:
%%sql

SELECT SUBSTR(Date, 6, 2) AS Month_date, Booster_Version, Launch_Site, Landing_Outcome
FROM SPACEXTBL
WHERE Landing_Outcome LIKE '%Failure%' AND Landing_Outcome LIKE '%drone%';

 * sqlite:///my_data1.db
Done.


,Month_date,Booster_Version,Launch_Site,Landing_Outcome
0,01,F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship)
1,04,F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship)
2,01,F9 v1.1 B1017,VAFB SLC-4E,Failure (drone ship)
3,03,F9 FT B1020,CCAFS LC-40,Failure (drone ship)
4,06,F9 FT B1024,CCAFS LC-40,Failure (drone ship)


##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [18]:
%%sql

SELECT Landing_Outcome, COUNT(Landing_Outcome)
FROM SPACEXTBL
WHERE SUBSTR(Date, 0, 12) > SUBSTR('2010-06-04', 0, 12)  AND SUBSTR(Date, 0, 12) < SUBSTR('2017-03-20', 0, 12)
GROUP BY Landing_Outcome
ORDER BY COUNT(Landing_Outcome) DESC;

 * sqlite:///my_data1.db
Done.


,Landing_Outcome,COUNT(Landing_Outcome)
0,No attempt,10
1,Success (drone ship),5
2,Failure (drone ship),5
3,Success (ground pad),3
4,Controlled (ocean),3
5,Uncontrolled (ocean),2
6,Precluded (drone ship),1
7,Failure (parachute),1


In [19]:
con.close()